In [11]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.metrics import classification_report
from tqdm import tqdm
from time import time
import numpy as np

In [12]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train=x_train.reshape(60000,784)
x_test=x_test.reshape(10000,784)

data=np.zeros([70000,784])
data[:60000,:]=x_train
data[60000:,:]=x_test

label=np.zeros(70000)
label[:60000]= y_train
label[60000:]= y_test

train_filter = np.where((y_train == 1 ) | (y_train == 2))
test_filter = np.where((y_test == 1) | (y_test == 2))

In [13]:
x_train, y_train = x_train[train_filter], y_train[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]

In [6]:
def naive_bayes(data,label):

    n_s,n_f=data.shape
    classes=np.unique(label)
    n_c=len(classes)

    total_data=np.zeros([n_s,n_f+1])
    total_data[:,:-1]=data
    total_data[:,-1]=label

    np.random.shuffle(total_data)
    trainX=total_data[:60000,:]

    np.random.shuffle(trainX)
    testX=total_data[60000:,:]

    np.random.shuffle(testX)
    testX_c=testX[:,:-1]
    testX_l=testX[:,-1]

    mean_v=np.zeros([n_c,n_f])
    var_v=np.zeros([n_c,n_f])

    c_prob=[]
    confusion_matrix=np.zeros([n_c,n_c])

    d_acc=[]

    for c in classes:
        trainX_c=trainX[trainX[:,-1]==c]
        trainX_c=trainX_c[:,:-1]
        c_prob.append(len(trainX_c)/len(trainX))
        mean_v[int(c),:]=trainX_c.mean(axis=0)
        var_v[int(c),:]=trainX_c.var(axis=0)

    var_v=var_v+1000
    count=0

    for i in range(testX.shape[0]):
        lists=[]
        for j in range(n_c):
            numerator=np.exp(-((testX_c[i]-mean_v[j])**2)/(2*var_v[j]))
            denominator=np.sqrt(2*np.pi*(var_v[j]))
            prob_xc=numerator/denominator
            ratio=np.sum(np.log(prob_xc))
            lists.append(ratio)
        pred=lists.index(max(lists))
        if pred == testX_l[i]:
            count=count+1
            confusion_matrix[int(testX_l[i])][int(testX_l[i])]=confusion_matrix[int(testX_l[i])][int(testX_l[i])]+1
        else:
            for k in range(n_c):
                if pred == k:
                    confusion_matrix[int(testX_l[k])][int(testX_l[i])]=confusion_matrix[int(testX_l[k])][int(testX_l[i])]+1
    for l in classes:
        check=testX[testX[:,-1]==l]
        a=(confusion_matrix[int(l)][int(l)])/check.shape[0]
        d_acc.append(a)
    o_acc=count/testX.shape[0]
    return(d_acc,o_acc,confusion_matrix,mean_v,var_v)

In [16]:
# Вызов классификатор из Naive Bayes Function
(digit_accuracy,overall_accuracy,matrix,mean_v,var_v)=naive_bayes(data,label)

In [17]:
digit=['1', '2']
naive_df = pd.DataFrame(list(zip(digit, digit_accuracy)), columns =['Digit','Digit Accuracy'])
print('Digit (Individual Class) Accuracy of the Samples:')
naive_df

Digit (Individual Class) Accuracy of the Samples:


,Digit,Digit Accuracy
0,1,0.899092
1,2,0.972801


In [20]:
print('Overall Accuracy of Naive Bayes Model: '+str(overall_accuracy))
overall_accuracy

Overall Accuracy of Naive Bayes Model: 0.7975


0.7975